In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
train=pd.read_csv('Dataset.csv')

In [3]:
train=train.rename(columns={
    'Unnamed: 0':'idx', 
   
    'NUM_Comments':'Num_Comments',
    
    'Coronavirus':'Label1Coronovirus', 
    'Politics':'Label2Politics', 
    'Non-Political':'Label3Non-Political',
    'Unnamed: 11':'Label4None', 
    '| Repost |':'Label5Repost', 
    'Unverified':'Label6Unverified', 
    'Policy/Economy':'Label7Policy/Economy',
    'Photography':'Label8Photography', 
    'Business/Finance':'Label9Buisness/Finance', 
    'AskIndia':'Label10AskIndia',
    'Scheduled':'Label11Scheduled',
    '| Stickied Topic |':'Label12Stickied-Topic', 
    '| Unverified Content / Disreputed Source |':'Label13Unverified-Content',
    'Science/Technology':'Label14Science/Technology', 
    'Food':'Label15Food', 
    '| Not Original/Relevant Title |':'Label16Not Original',
    '| Social Media Rules |':'Label17Social Media', 
    'CAA-NRC-NPR':'Label18CAA/NRC', 
    'Megathread':'Label19Megathread', 
    '🥳':'Label20Unk'})

In [4]:
train.columns

Index(['idx', 'Title', 'Score', 'ID', 'Image', 'Num_Comments', 'Created',
       'Body', 'Label1Coronovirus', 'Label2Politics', 'Label3Non-Political',
       'Label4None', 'Label5Repost', 'Label6Unverified',
       'Label7Policy/Economy', 'Label8Photography', 'Label9Buisness/Finance',
       'Label10AskIndia', 'Label11Scheduled', 'Label12Stickied-Topic',
       'Label13Unverified-Content', 'Label14Science/Technology', 'Label15Food',
       'Label16Not Original', 'Label17Social Media', 'Label18CAA/NRC',
       'Label19Megathread', 'Label20Unk'],
      dtype='object')

In [5]:
train['Label4None'].value_counts()

0    686
1      6
Name: Label4None, dtype: int64

In [6]:
train['Label1Coronovirus'].value_counts()

0    386
1    306
Name: Label1Coronovirus, dtype: int64

In [7]:
train['Label2Politics'].value_counts()

0    581
1    111
Name: Label2Politics, dtype: int64

Potentional error is uneven dataset


In [14]:
title_sents=[]
for i in range(train.shape[0]):
    title_sents.append(train.loc[i,'Title'])
    
        
title_sents

['Coronavirus (COVID-19) Megathread - News and Updates - 3',
 'Firozabad police fact-checking zee news (translation in comments)',
 'Donald Trump talks of ‘retaliation’ if India turns down Covid drug request',
 'Mom came up with this impressive idea to celebrate my bday amidst lockdown (choco pie & chena khir)',
 '9 PM 9 Minutes: House Catches Fire In Bihar; Grandmother, Two Girls Die',
 'Cartoon by Pencilashan',
 "Pandemic impact: Mukesh Ambani's net worth drops 28% to $48 billion in 2 months",
 'Symbiotic Relationship',
 'Coronavirus: How Shashi Tharoor Spent His MP Funds Just Before Centre Scrapped It',
 'Mainstream Indian media for you. Palestine is now in Africa.',
 "Raipur AIIMS denies 'spitting, misbehaviour' by Tablighi Jamaat patient",
 'ICMR prepares plan to increase testing capacity to 100,000 a day',
 "Viral audio: False conspiracy theory about Modi govt introducing 'vaccine' to kill Muslims - Alt News",
 'Explained: Sampling bias drove sensationalist reporting around Tabli

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext

import spacy

In [17]:
##CustomPunctuation with ?
import string
k=string.punctuation

punct=[]
for i in range(len(k)):
    if(k[i]=='?'):
        continue
    else: 
        punct.append(k[i])
    
punct

['!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~']

In [18]:
##This is the final working Tokenizer

spacy_eng=spacy.load('en')
nlp=spacy.load("en_core_web_sm")
def tokenize_eng(text):
    sent=[]
    lema=nlp(text)
    for tok in spacy_eng.tokenizer(text):
        
        word=tok.lemma_
        #word=tok.text
        
        if word not in punct:
            sent.append(word)
    
    return sent

In [20]:
nlp=spacy.load("en_core_web_sm")
#text=nlp(sents[0])
#print(sents)

for sent in title_sents:
    sent=tokenize_eng(sent)

In [24]:
title_sents[0]

'Coronavirus (COVID-19) Megathread - News and Updates - 3'

In [50]:
from torchtext.data import Field


TEXT=Field(sequential=True,tokenize=tokenize_eng,lower=True,batch_first=True)
LABEL=Field(sequential=False,use_vocab=False,batch_first=True)

In [51]:
from torchtext.data import TabularDataset

tv_field=[
         ('idx',None),
         ('Title',TEXT),
         ('Score',None),
         ('ID',None),
         ('Image',None),
         ('Num_Comments',None),
         ('Created',None),
         ('Body',None),
         ('Label1Coronovirus',LABEL),
         ('Label2Politics',LABEL),
         ('Label3Non-Political',LABEL),
         ('Label4None',LABEL),
         ('Label5Repost',LABEL),
         ('Label6Unverified',LABEL),
         ('Label6Policy/Economy',LABEL),
         ('Label7Photography',LABEL),
         ('Label8Buisness/Finance',LABEL),
         ('Label9AskIndia',LABEL),
         ('Label10Scheduled',LABEL),
         ('Label11Stickied-Topic',LABEL),
         ('Label12Unverified-Content',LABEL),
         ('Label13Science/Technology',LABEL),
         ('Label14Food',LABEL),
         ('Label15Not Original',LABEL),
         ('Label16Social Media',LABEL),
         ('Label17CAA/NRC',LABEL),
         ('Label18Megathread',LABEL),
         ('Label19Unk',LABEL)]
         

In [52]:
train=TabularDataset(path='Dataset.csv',
                    format='csv',
                    skip_header=True,
                    fields=tv_field)

In [53]:
print(train[0].__dict__.keys())

dict_keys(['Title', 'Label1Coronovirus', 'Label2Politics', 'Label3Non-Political', 'Label4None', 'Label5Repost', 'Label6Unverified', 'Label6Policy/Economy', 'Label7Photography', 'Label8Buisness/Finance', 'Label9AskIndia', 'Label10Scheduled', 'Label11Stickied-Topic', 'Label12Unverified-Content', 'Label13Science/Technology', 'Label14Food', 'Label15Not Original', 'Label16Social Media', 'Label17CAA/NRC', 'Label18Megathread', 'Label19Unk'])


In [54]:
train[0].Title
print(train[0].Label1Coronovirus,train[0].Label2Politics)

1 0


In [55]:
TEXT.build_vocab(train,min_freq=1)

In [56]:
from torchtext.data import Iterator, BucketIterator

train_iterator=BucketIterator(train,
                             batch_size=32,
                             device=-1,
                             sort_key=lambda x:len(x.Title),
                             sort_within_batch=False,
                             repeat=False)



The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [60]:
for example in enumerate(train_iterator):
    print(example[1].Title.shape)
    print(example[1].Label1Coronovirus.shape)
    break

torch.Size([32, 43])
torch.Size([32])


In [61]:
train[100].Title

['medical',
 'worker',
 'be',
 'target',
 'for',
 'raise',
 'issue',
 'of',
 'protective',
 'gear',
 'say',
 'aiims',
 'rda']

In [90]:
a=list(train[0].__dict__.keys())
labels=a[1:]
print(labels)

['Label1Coronovirus', 'Label2Politics', 'Label3Non-Political', 'Label4None', 'Label5Repost', 'Label6Unverified', 'Label6Policy/Economy', 'Label7Photography', 'Label8Buisness/Finance', 'Label9AskIndia', 'Label10Scheduled', 'Label11Stickied-Topic', 'Label12Unverified-Content', 'Label13Science/Technology', 'Label14Food', 'Label15Not Original', 'Label16Social Media', 'Label17CAA/NRC', 'Label18Megathread', 'Label19Unk']


In [98]:

class BatchWrapper:
    def __init__(self,dat,x_var,*y_vars):
        self.dat=dat
        self.x_vars=x_var
        self.y_vars=y_vars
        
    def __iter__(self):
        for batch in self.dat:
            x=getattr(batch,self.x_vars)
            y=torch.cat([getattr(batch,y).unsqueeze(1) for y in self.y_vars[0]],dim=1).float()
            
            yield(x,y)

    def __len__(self):
        return len(self.dat)
        
train_dat=BatchWrapper(train_iterator,'Title',labels)

In [99]:
next(train_dat.__iter__())

(tensor([[ 912, 1991,    3,  ...,    1,    1,    1],
         [  57,  897,   13,  ...,    1,    1,    1],
         [ 293,   25,   91,  ...,    1,    1,    1],
         ...,
         [ 500,    5,  596,  ...,    1,    1,    1],
         [  26,    6, 1534,  ...,    1,    1,    1],
         [1742,    4, 1215,  ...,    1,    1,    1]]),
 tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.

In [104]:
for example in enumerate(train_dat):
    print(example[1][0].shape)
    print(example[1][1].shape)
    print(example[0])
    break
    
#The y is 20 dimensinal and x varies

torch.Size([32, 22])
torch.Size([32, 20])
0


In [219]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

class Model(nn.Module):
    def __init__(self,n_vocab,
                batch_size=32,
                embedding_dim=100,
                hidden_dim=64,
                num_layer=1,
                 dropout=0.3,
                bidirectional=False):
        super(Model,self).__init__()
        self.n_vocab=n_vocab
        self.batch_size=batch_size
        self.embedding_dim=embedding_dim
        self.hidden_dim=hidden_dim
        self.num_layer=num_layer
        self.bidirectional=bidirectional
        self.dropout=dropout
        
        
        self.embedding=nn.Embedding(n_vocab,embedding_dim)
        self.rnn=nn.RNN(self.embedding_dim, self.hidden_dim,
                       num_layers=self.num_layer,
                       batch_first=True)
        self.fc1=nn.Linear(hidden_dim,20)
        
        self.dropout=nn.Dropout(dropout)
        
    def forward(self,data):
        
            
        embedded=self.dropout(self.embedding(data))
        output,hidden=self.rnn(embedded)
        
       # print(f"Hidden Dim {hidden.size()}")
        output=F.sigmoid(self.fc1(hidden))
        
        return output

In [220]:
ENC_EMB_DIM=256
DEC_EMB_DIM=256
HID_DIM=512
ENC_DROPOUT=0.5
DEC_DROPOUT=0.5

Input_Vocab=len(TEXT.vocab)

model=Model(n_vocab=Input_Vocab,embedding_dim=ENC_EMB_DIM,hidden_dim=HID_DIM,
           dropout=ENC_DROPOUT)

print(model)

Model(
  (embedding): Embedding(2734, 256)
  (rnn): RNN(256, 512, batch_first=True)
  (fc1): Linear(in_features=512, out_features=20, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [158]:
len(TEXT.vocab)


2734

In [221]:

import tqdm
import torch.optim as optim
optimizer=optim.Adam(model.parameters())

criterion=nn.BCEWithLogitsLoss()
EPOCH=2

In [227]:
for x,y in tqdm.tqdm(train_dat):
    print(x.shape)
    print(y.shape)
    pred=model(x)
    break
    

a=pred.squeeze(0)
b=y

  0%|          | 0/22 [00:00<?, ?it/s]

torch.Size([32, 58])
torch.Size([32, 20])


In [233]:
a[31]

tensor([0.4690, 0.5022, 0.5425, 0.4864, 0.4850, 0.5229, 0.5825, 0.5364, 0.5220,
        0.5827, 0.5796, 0.5110, 0.6235, 0.4443, 0.4291, 0.4913, 0.4087, 0.4288,
        0.6781, 0.3700], grad_fn=<SelectBackward>)

In [161]:
#TEXT.vocab.itos[949],TEXT.vocab.itos[311],TEXT.vocab.itos[27],TEXT.vocab.itos[1854]
len(train)

TypeError: object of type 'function' has no len()

In [245]:
def train(model,train_iter):
    for epoch in range(1,20):
        running_loss=0.0
        running_corrects=0
        model.train()
        epoch_lo=0
        for x,y in tqdm.tqdm(train_iter):
            optimizer.zero_grad()
            preds=model(x)
            
            #print(f"y: {y.shape}")
            preds=preds.squeeze(0)
            
            
            #print(f"Hello {preds.shape}, {y.shape}")
            for i in range(len(preds)):
                k=preds[i].detach().numpy()
                d=y[i].detach().numpy()
                if(np.argmax(k)==np.argmax(d)):
                    epoch_lo+=1
            #print(preds)
            #print(preds.shape)
            loss=criterion(y,preds)
            loss.backward()
            optimizer.step()
            
            running_loss+=loss.item()*x.size(0)
        epoch_loss=running_loss/692
        print(f'Epoch :{epoch}, Training Loss: {epoch_loss}, Num_Correct={epoch_lo}')

In [246]:
train(model,train_dat)

  5%|▍         | 1/22 [00:00<00:02,  8.66it/s]

Epoch :1, Training Loss: 0.674722561257423, Num_Correct=294


  9%|▉         | 2/22 [00:00<00:01, 11.50it/s]

Epoch :2, Training Loss: 0.6749637171712225, Num_Correct=292


  9%|▉         | 2/22 [00:00<00:01, 14.95it/s]

...

Epoch :17, Training Loss: 0.6750787544801745, Num_Correct=221


  5%|▍         | 1/22 [00:00<00:02,  8.30it/s]

Epoch :18, Training Loss: 0.6750709996747144, Num_Correct=278


100%|██████████| 22/22 [00:01<00:00, 13.63it/s]

Epoch :19, Training Loss: 0.6748671262939542, Num_Correct=280


In [201]:
a=torch.zeros((2,10),requires_grad=True)
b=torch.rand((2,10))
a[0,5]=1
a[1,6]=1

b[0,5]=10
b[1,6]=10
print(a,b)

k=a[1].detach().numpy()
d=b[1].detach().numpy()
import numpy
print(np.argmax(k))
print(np.argmax(d))
print(a)

tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], grad_fn=<CopySlices>) tensor([[3.9022e-01, 7.3591e-01, 2.4129e-01, 3.9507e-01, 2.3940e-02, 1.0000e+01,
         9.0931e-02, 8.5006e-01, 4.3227e-01, 9.6372e-01],
        [6.5665e-01, 4.4008e-03, 4.4291e-01, 8.9158e-01, 4.6105e-01, 3.9616e-01,
         1.0000e+01, 2.0257e-01, 7.8120e-01, 2.8559e-01]])
6
6
tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]], grad_fn=<CopySlices>)


In [244]:
692%32

20

In [247]:
a="Is coronavirus killed by mosquitos?"
a=tokenize_eng(a)

In [249]:
se=[]
for word in a:
    se.append(TEXT.vocab.stoi[word])

In [255]:
se=torch.tensor(se)
se=se.unsqueeze(0)
se.shape

/home/ishan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


torch.Size([1, 6])

In [256]:
output=model(se)

In [257]:
output

tensor([[[0.9998, 0.9999, 0.9982, 0.9993, 0.9997, 0.9810, 0.1274, 0.9993,
          0.9966, 0.9999, 0.9994, 0.9986, 0.9917, 0.9978, 0.9710, 0.9999,
          0.9999, 0.9968, 0.9998, 0.9960]]], grad_fn=<SigmoidBackward>)

In [263]:
np.argmax(output[0][0].detach())

tensor(1)